In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re, pickle
le = LabelEncoder()
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

C:\Users\SRAVANTHI SHOROFF\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10051] A socket operation was attempted to an unreachable network
  RuntimeWarning,
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
df = pd.read_excel(r'SCOM2016_Alerts_Updated1.xlsx')
df.head()

,Server name,Summary of the alert,Description,Priority,Severity,Category,TimeRaised,TimeAdded,Type of the Alert (True or False),Action to be taken,Ignorable or Non-Ignorable
0,NaN,Veeam VMware: Scheduled Task Failed,[ScheduledTaskFailedEvent] Task VMware vSphere...,Normal,Warning,EventCollection,2018-10-10 19:44:48,2018-10-10 19:44:48,True,Can be ignored,Ignorable
1,UKMLWSQL904,Workflow Initialization: Failed to start a wor...,"Data was found in the output, but has been dro...",Normal,Warning,EventCollection,2019-04-09 22:56:48,2019-04-09 22:56:48,True,Can be ignored,Ignorable
2,UKMLWSQL904,MSSQL 2016: Discovery failed,Event ID: 7105. Management Group: TelefonicaUK...,Normal,Error,Alert,2019-04-09 22:56:48,2019-04-09 22:56:48,True,mail to DB team,Non-ignorable
3,UKMLWSQL903,Workflow Initialization: Failed to start a wor...,"Data was found in the output, but has been dro...",Normal,Warning,EventCollection,2019-04-09 22:56:57,2019-04-09 22:56:57,True,Can be ignored,Ignorable
4,UKMLWSQL903,MSSQL 2016: Monitoring warning,Event ID: 4211. Management Group: TelefonicaUK...,Normal,Warning,Alert,2019-04-09 23:05:14,2019-04-09 23:05:14,True,mail to DB team,Non-ignorable


In [3]:
df.shape

(5318, 11)

In [4]:
df.keys()

Index(['Server name', 'Summary of the alert', 'Description', 'Priority',
       'Severity', 'Category', 'TimeRaised', 'TimeAdded',
       'Type of the Alert (True or False)', 'Action to be taken ',
       'Ignorable or Non-Ignorable'],
      dtype='object')

In [5]:
df1 = df[['Summary of the alert','Ignorable or Non-Ignorable']]

In [6]:
df1.shape

(5318, 2)

In [7]:
df1.keys()

Index(['Summary of the alert', 'Ignorable or Non-Ignorable'], dtype='object')

In [8]:
df1['Ignorable or Non-Ignorable'].value_counts()

Non-ignorable    4199
Ignorable        1119
Name: Ignorable or Non-Ignorable, dtype: int64

In [9]:
df1['Ignorable or Non-Ignorable']=df1['Ignorable or Non-Ignorable'].apply(lambda x: 1 if (x)=="Non-ignorable" else 0)

In [10]:
df1['Ignorable or Non-Ignorable'].value_counts()

1    4199
0    1119
Name: Ignorable or Non-Ignorable, dtype: int64

In [11]:
df1['Summary of the alert'].value_counts()

Health Service Heartbeat Failure                                                  2124
Clean Shutdown Initiated                                                           559
Failed to Connect to Computer                                                      342
Dell OMSS Controller event log (3)                                                 189
Cluster resource group offline or partially online                                 130
                                                                                  ... 
Dell OMSS Controller write policy has been changed to Write Back                     1
Management point relaymgr backlog alert                                              1
Operations Manager Web Console Unavailable                                           1
SSRS 2012: An Error occurred during execution of a SSRS 2012 MP managed module       1
Alert Parameter Replacement Failure                                                  1
Name: Summary of the alert, Length: 190, dt

In [12]:
df1['Summary of the alert'].dropna(how='all')

0                     Veeam VMware: Scheduled Task Failed
1       Workflow Initialization: Failed to start a wor...
2                            MSSQL 2016: Discovery failed
3       Workflow Initialization: Failed to start a wor...
4                          MSSQL 2016: Monitoring warning
                              ...                        
5313                             Clean Shutdown Initiated
5314                             Clean Shutdown Initiated
5315    MSSQL on Windows: Buffer Cache Hit Ratio is to...
5316             Dell OMSS Controller battery is charging
5317       Dell OMSS The battery charge cycle is complete
Name: Summary of the alert, Length: 5318, dtype: object

In [13]:
df1['Ignorable or Non-Ignorable'].dropna(how='all')

0       0
1       0
2       1
3       0
4       1
       ..
5313    1
5314    1
5315    1
5316    0
5317    0
Name: Ignorable or Non-Ignorable, Length: 5318, dtype: int64

In [14]:
df1['Summary of the alert'] = df1['Summary of the alert'].apply(lambda x: x.lower())
# df['Alert'] = df['Alert'].apply(lambda x: x.lower())

In [15]:
df1['Summary of the alert'] = df1['Summary of the alert'].apply(lambda s: re.sub(r"[^a-zA-Z0-9]"," ",s))

In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [17]:
lemmatizer=WordNetLemmatizer()
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens =[w for w in tokens if not w in stop_words] # [w for w in
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokens])
    return lemmatized_output

In [18]:
df1['Summary of the alert'].apply(tokenize)

0                      veeam vmware scheduled task failed
1       workflow initialization failed start workflow ...
2                             mssql 2016 discovery failed
3       workflow initialization failed start workflow ...
4                           mssql 2016 monitoring warning
                              ...                        
5313                             clean shutdown initiated
5314                             clean shutdown initiated
5315              mssql window buffer cache hit ratio low
5316                dell omss controller battery charging
5317              dell omss battery charge cycle complete
Name: Summary of the alert, Length: 5318, dtype: object

In [22]:
tfidf_vector=TfidfVectorizer(use_idf=True,max_features=1000)

In [29]:
tf = tfidf_vector.fit_transform(df1['Summary of the alert']).todense()

In [31]:
tf

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
pickle.dump(tf, open("vectorized.pkl", "wb"))

In [36]:
vectorizer = pickle.load(open("vectorized.pkl", "rb"))

In [37]:
vectorizer

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [38]:
#taking X and y
X = vectorizer
y = df1['Ignorable or Non-Ignorable']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [40]:
glmMod = LogisticRegression()
glmMod.fit(X_train, y_train)
y_pred=glmMod.predict(X_test)
print("\n\n***** Logistic Regression classification report:*****\n")
print(classification_report(y_test, y_pred))



***** Logistic Regression classification report:*****

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       287
           1       0.99      1.00      0.99      1043

    accuracy                           0.99      1330
   macro avg       0.99      0.97      0.98      1330
weighted avg       0.99      0.99      0.99      1330



In [82]:
filename = 'logisticregressionMod.pkl'
pickle.dump(glmMod, open('logisticregressionMod.pkl', 'wb'))

In [83]:
rfMod = RandomForestClassifier()
rfMod.fit(X_train, y_train)
y_pred=rfMod.predict(X_test)
print("\n\n***** RandomForestClassifier report:*****\n")
print(classification_report(y_test, y_pred))



***** RandomForestClassifier report:*****

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       283
           1       1.00      1.00      1.00      1047

    accuracy                           1.00      1330
   macro avg       1.00      0.99      0.99      1330
weighted avg       1.00      1.00      1.00      1330



In [84]:
filename = 'randomforestMod.pkl'
pickle.dump(rfMod, open('randomforestMod.pkl', 'wb'))

In [85]:
gbMod = GradientBoostingClassifier()
gbMod.fit(X_train, y_train)
y_pred=gbMod.predict(X_test)
print("\n\n***** GradientBoosting report:*****\n")
print(classification_report(y_test, y_pred))



***** GradientBoosting report:*****

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       283
           1       0.99      1.00      0.99      1047

    accuracy                           0.99      1330
   macro avg       0.99      0.98      0.99      1330
weighted avg       0.99      0.99      0.99      1330



In [86]:
filename = 'gradientboostMod.pkl'
pickle.dump(gbMod, open('gradientboostMod.pkl', 'wb'))

In [87]:
adaMod = AdaBoostClassifier()
adaMod.fit(X_train, y_train)
y_pred = adaMod.predict(X_test)
print("\n\n***** Adaboost report:*****\n")
print(classification_report(y_test, y_pred))



***** Adaboost report:*****

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       283
           1       1.00      1.00      1.00      1047

    accuracy                           1.00      1330
   macro avg       1.00      1.00      1.00      1330
weighted avg       1.00      1.00      1.00      1330



In [88]:
filename = 'adaboostMod.pkl'
pickle.dump(adaMod, open('adaboostMod.pkl', 'wb'))

In [89]:
from sklearn.ensemble import VotingClassifier
votingMod = VotingClassifier(estimators=[('LogisticRegression', glmMod),('RandomForrest', rfMod),("GradientBoosting",gbMod),("AdaBoost",adaMod)], voting='soft')
votingMod = votingMod.fit(X_train, y_train)
test_labels=votingMod.predict((X_test))
#votingMod.score(X_test_transform, y_test)

print("\n\n***** Model Ensemble score report:*****\n")
print(classification_report(y_test, test_labels))




***** Model Ensemble score report:*****

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       283
           1       1.00      1.00      1.00      1047

    accuracy                           1.00      1330
   macro avg       1.00      0.99      0.99      1330
weighted avg       1.00      1.00      1.00      1330



In [90]:
filename = 'votingclassifierMod.pkl'
pickle.dump(votingMod, open('votingclassifierMod.pkl', 'wb'))

In [91]:
len(test_labels)

1330

In [92]:
pwd

'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_ML'

In [40]:
pkl_file = 'C:\\Users\\SRAVANTHI SHOROFF\\Desktop\\sravanthi\\SCOM_ML\\votingclassifierMod.pkl'
with open(pkl_file,'rb') as file:
        votingclassifier = pickle.load(file)

In [41]:
# predict class
y_pred = votingclassifier.predict(X)
print(y_pred)

[0 0 1 ... 1 0 0]


In [43]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1119
           1       1.00      1.00      1.00      4199

    accuracy                           1.00      5318
   macro avg       1.00      0.99      1.00      5318
weighted avg       1.00      1.00      1.00      5318



In [44]:
output = pd.DataFrame(columns=['y_pred'],data = y_pred)

In [50]:
output['y_pred'].value_counts()

1    4213
0    1105
Name: y_pred, dtype: int64

In [52]:
output['pred_val'] = output['y_pred'].replace(to_replace=1,value='Yes')

In [53]:
output['pred_val'].value_counts()

Yes    4213
0      1105
Name: pred_val, dtype: int64

In [54]:
false_output = output[output['pred_val']==0]

In [55]:
len(false_output)

1105